In [ ]:
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 52.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.5 MB/s eta 0:00:00


In [ ]:
!pip install pyngrok

In [ ]:
!npm install localtunnel

npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm notice created a lockfile as package-lock.json. You should commit this file.
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ localtunnel@2.0.2
added 22 packages from 22 contributors and audited 22 packages in 2.528s

3 packages are looking for funding
  run `npm fund` for details

found 1 moderate severity vulnerability
  run `npm audit fix` to fix them, or `npm audit` for details


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%%writefile app.py
import streamlit as st
import pandas as pd
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px
from prophet import Prophet
from prophet.plot import plot_plotly, plot_components_plotly
import plotly.graph_objs as go
import warnings
warnings.filterwarnings('ignore')

@st.cache_data
def load_data():
    data = pd.read_csv("/content/drive/MyDrive/datasets/DailyDelhiClimateTrain_delhi_2017.csv")
    data["date"] = pd.to_datetime(data["date"], format='%Y-%m-%d')
    data['year'] = data['date'].dt.year
    data["month"] = data["date"].dt.month
    return data

@st.cache_data
def load_train():
    train=pd.read_excel("/content/drive/MyDrive/datasets/AQI_ttnagar_Epics.xlsx")
    train['Date'] = pd.to_datetime(train['Date'])
    train = train.drop(["AQI Status"], axis = 1)
    train = train.drop(["Benzene (µg/m3)"], axis = 1)
    train = train.dropna()
    return train

def page1(data):
    st.header("This page contains temperature, humidity, and wind speed plots.")

    # Mean temperature plot
    st.subheader("Mean Temperature Over the Years")
    mean_temp_overyears(data)

    # Humidity plot
    st.subheader("Humidity Over the Years")
    humidity_overyears(data)

    # Wind speed plot
    st.subheader("Wind Speed Over the Years")
    wind_speed_overyears(data)

def page2():
    st.header("Historical Observations on AQI of TT Nagar Bhopal-")

    # AQI over time
    st.subheader("AQI Overtime")
    aqi_otm(train)

    # AQI over the years
    st.subheader("AQI Over the years")
    aqi_oty(train)

def page3():
    st.header("AQI Prediction")
    predictions_aqi = aqi_prediction(train)
    aqi_pred_specific_date(predictions_aqi)

def page4():
  st.header("Weather Prediction")

  predictions_meantemp = weather_mt_prediction(data)
  specific_date_prediction_mt(predictions_meantemp)

  predictions_windspeed = weather_ws_prediction(data)
  specific_date_prediction_ws(predictions_windspeed)

  prediction_humidity = weather_hd_prediction(data)
  specific_date_prediction_hd(prediction_humidity)


#Weather Hist plots*************************************************************
@st.cache_data
def mean_temp_overyears(data):
    plt.style.use('fivethirtyeight')
    fig, ax = plt.subplots(figsize=(15, 10))
    plt.title("Temperature Change in Delhi Over the Years")
    sns.set(rc={'text.color': 'white'})
    sns.lineplot(data=data, x='month', y='meantemp', hue='year')
    st.pyplot(fig)

@st.cache_data
def wind_speed_overyears(data):
    plt.style.use('fivethirtyeight')
    fig, ax = plt.subplots(figsize=(15, 10))
    plt.title("Humidity Change in Delhi Over the Years")
    sns.set(rc={'text.color': 'white'})
    sns.lineplot(data=data, x='month', y='humidity', hue='year')
    st.pyplot(fig)

@st.cache_data
def humidity_overyears(data):
    plt.style.use('fivethirtyeight')
    fig, ax = plt.subplots(figsize=(15, 10))
    plt.title("Wind speed Change in Delhi Over the Years")
    sns.set(rc={'text.color': 'white'})
    sns.lineplot(data=data, x='month', y='wind_speed', hue='year')
    st.pyplot(fig)

#Weather Prediction*************************************************************

#Meantemp***********************************
def weather_mt_prediction(data):
    forecast_data_meanTemp = data.rename(columns={"date": "ds", "meantemp": "y"})
    model = Prophet()
    model.fit(forecast_data_meanTemp)
    forecasts_meanTemp = model.make_future_dataframe(periods=365)
    predictions_meantemp = model.predict(forecasts_meanTemp)
    fig = plot_plotly(model, predictions_meantemp)
    fig.add_trace(go.Scatter(x=forecast_data_meanTemp['ds'], y=forecast_data_meanTemp['y'],
                             mode='lines', name='Observed'))
    fig.add_trace(go.Scatter(x=predictions_meantemp['ds'], y=predictions_meantemp['yhat'],
                             mode = 'lines', name = 'Prediction'))

    # Customize layout
    fig.update_layout(title='Mean Temperature Forecast',
                      xaxis_title='Date',
                      yaxis_title='Mean Temperature')
    st.plotly_chart(fig)
    return predictions_meantemp


def specific_date_prediction_mt(predictions_meantemp):
    specific_date_input = st.text_input("Enter the date (YYYY-MM-DD):")
    if specific_date_input:
        specific_date = pd.to_datetime(specific_date_input)
        specific_date_prediction = predictions_meantemp[predictions_meantemp['ds'] == specific_date]

        # Check if specific_date_prediction is not None before accessing its attributes
        if specific_date_prediction is not None and not specific_date_prediction.empty:
            st.write("Mean Temperature Prediction for {} is : {}".format(specific_date, specific_date_prediction['yhat'].values[0]))
        else:
            st.write("No prediction available for the specified date.")
#*******************************************

#Wind Speed
def weather_ws_prediction(data):
  forecast_data_windspeed = data.rename(columns = {"date": 'ds', "wind_speed": 'y'})
  model = Prophet()
  model.fit(forecast_data_windspeed)
  forecasts_windspeed = model.make_future_dataframe(periods=365)
  predictions_windspeed = model.predict(forecasts_windspeed)
  fig = plot_plotly(model, predictions_windspeed)
  fig.add_trace(go.Scatter(x=forecast_data_windspeed['ds'], y=forecast_data_windspeed['y'],
                             mode='lines', name='Observed'))
  fig.add_trace(go.Scatter(x=predictions_windspeed['ds'], y=predictions_windspeed['yhat'],
                             mode = 'lines', name = 'Prediction'))

    # Customize layout
  fig.update_layout(title='Wind Speed Forecast',
                      xaxis_title='Date',
                      yaxis_title='Wind Speed')
  st.plotly_chart(fig)
  return predictions_windspeed

def specific_date_prediction_ws(predictions_windspeed):
  specific_date_input = st.text_input("Enter the date (YYYY-MM-DD):", key="specific_date_input_ws")
  if specific_date_input:
        specific_date = pd.to_datetime(specific_date_input)
        specific_date_prediction = predictions_windspeed[predictions_windspeed['ds'] == specific_date]

        # Check if specific_date_prediction is not None before accessing its attributes
        if specific_date_prediction is not None and not specific_date_prediction.empty:
            st.write("Wind Speed Prediction for {} is : {}".format(specific_date, specific_date_prediction['yhat'].values[0]))
        else:
            st.write("No prediction available for the specified date.")

#Humidity***********************************************************************
def weather_hd_prediction(data):
  forecast_data_humidity = data.rename(columns = {"date": 'ds', "humidity": 'y'})
  model = Prophet()
  model.fit(forecast_data_humidity)
  forecasts_humidity = model.make_future_dataframe(periods=365)
  predictions_humidity = model.predict(forecasts_humidity)
  fig = plot_plotly(model, predictions_humidity)
  fig.add_trace(go.Scatter(x=forecast_data_humidity['ds'], y=forecast_data_humidity['y'],
                             mode='lines', name='Observed'))
  fig.add_trace(go.Scatter(x=predictions_humidity['ds'], y=predictions_humidity['yhat'],
                             mode = 'lines', name = 'Prediction'))

    # Customize layout
  fig.update_layout(title='Humidity Forecast',
                      xaxis_title='Date',
                      yaxis_title='Humidity')
  st.plotly_chart(fig)
  return predictions_humidity

def specific_date_prediction_hd(predictions_humidity):
  specific_date_input = st.text_input("Enter the date (YYYY-MM-DD):", key="specific_date_input_hd")
  if specific_date_input:
        specific_date = pd.to_datetime(specific_date_input)
        specific_date_prediction = predictions_humidity[predictions_humidity['ds'] == specific_date]

        # Check if specific_date_prediction is not None before accessing its attributes
        if specific_date_prediction is not None and not specific_date_prediction.empty:
            st.write("Humidity Prediction for {} is : {}".format(specific_date, specific_date_prediction['yhat'].values[0]))
        else:
            st.write("No prediction available for the specified date.")



#AQI Hist plots*****************************************************************

@st.cache_data
def aqi_otm(train):

    x = train["Date"].tolist()  # Convert Series to list
    y = train["AQI No."].tolist()
    figure = px.line(data, x, y, title='AQI in TT Nagar Over the Years')
    st.plotly_chart(figure)

@st.cache_data
def aqi_oty(train):
    train["Date"] = pd.to_datetime(train["Date"], format = '%Y-%m-%d')
    train['year'] = train['Date'].dt.year
    train["month"] = train["Date"].dt.month
    plt.style.use('fivethirtyeight')
    fig, ax = plt.subplots(figsize=(15, 10))
    plt.title("AQI Change in Bhopal(TT Nagar) Over the Years")
    sns.lineplot(data=train, x='month', y='AQI No.', hue='year')
    st.pyplot(fig)


#AQI Prediction*****************************************************************

@st.cache_data
def aqi_prediction(train):
    forecast_AQI = train.rename(columns={"Date": "ds", "AQI No.": "y"})
    model = Prophet()
    model.fit(forecast_AQI)
    forecasts_aqi = model.make_future_dataframe(periods=365)
    predictions_aqi = model.predict(forecasts_aqi)

    # Plotting observed data and predictions
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=forecast_AQI['ds'], y=forecast_AQI['y'], mode='lines', name='Observed'))
    fig.add_trace(go.Scatter(x=predictions_aqi['ds'], y=predictions_aqi['yhat'], mode='lines', name='Predictions'))

    fig.update_layout(title='AQI Prediction',
                      xaxis_title='Date',
                      yaxis_title='AQI')

    # Show the Plotly figure using st.plotly_chart()
    st.plotly_chart(fig)

    return predictions_aqi


def aqi_pred_specific_date(predictions_aqi):
    # Obtain prediction for a specific date
    st.write("Enter the date (YYYY-MM-DD): ")
    specific_date_input = st.text_input("")

    if specific_date_input:
        specific_date = pd.to_datetime(specific_date_input)
        specific_date_prediction = predictions_aqi[predictions_aqi['ds'] == specific_date]

        # Print the prediction for the specific date
        if not specific_date_prediction.empty:
            st.write("AQI Prediction for {} is : {}".format(specific_date, specific_date_prediction['yhat'].values[0]))
        else:
            st.write("No prediction available for the specified date.")




# Load data*********************************************************************
data = load_data()
train = load_train()

# Sidebar navigation
page = st.sidebar.selectbox("Select Page", ["Historical Weather Data (Delhi)", "Historical AQI Data (TT Nagar Bhopal)", "AQI Predictor", "Weather Predictor(Currently only Delhi)"])

# Render selected page
if page == "Historical Weather Data (Delhi)":
    page1(data)
elif page == "Historical AQI Data (TT Nagar Bhopal)":
    page2()
elif page == "AQI Predictor":
    page3()
elif page == "Weather Predictor(Currently only Delhi)":
    page4()


Overwriting app.py


In [ ]:
!streamlit run /content/app.py &>/content/logs.txt &

In [ ]:
!streamlit run --server.port 8501 /content/app.py



2024-05-05 18:18:12.218 Port 8501 is already in use


In [ ]:
from pyngrok import ngrok


# Authenticate with your ngrok authentication token
ngrok.set_auth_token("2g2ia8R2rg72EYOS0esPmNyCrSj_33APXDdFZig3YzPFzt5xg")
# Set up a tunnel to the Streamlit app running on port 8501
public_url = ngrok.connect(addr='8501')
# Print the public URL
print("Public URL:", public_url)


Public URL: NgrokTunnel: "https://142a-34-125-167-146.ngrok-free.app" -> "http://localhost:8501"


In [ ]:
#import urllib
#print("Password/Enpoint IP for localtunnel is:",urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))

Password/Enpoint IP for localtunnel is: 34.73.139.230


In [ ]:
#!npx localtunnel --port 8501

npx: installed 22 in 1.987s
your url is: https://moody-rice-go.loca.lt
^C


In [ ]:
%%writefile app.py
import streamlit as st
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

@st.cache_data
def load_data():
    data = pd.read_csv("/content/drive/MyDrive/datasets/DailyDelhiClimateTrain_delhi_2017.csv")
    data["date"] = pd.to_datetime(data["date"], format='%Y-%m-%d')
    data['year'] = data['date'].dt.year
    data["month"] = data["date"].dt.month
    return data
@st.cache_data
def load_train():
  train = pd.read_excel("/content/drive/MyDrive/datasets/AQI_ttnagar_Epics.xlsx")
  train['Date'] = pd.to_datetime(train['Date'])
  train = train.drop(["AQI Status"], axis = 1)
  train = train.drop(["Benzene (µg/m3)"], axis = 1)
  train = train.dropna()
  train["Date"] = pd.to_datetime(train["Date"], format = '%Y-%m-%d')
  train['year'] = train['Date'].dt.year
  train["month"] = train["Date"].dt.month
  return train


#PAGE 1*************************************************************************

def page1(data):
    st.header("Weather feature changes over the years -")
    st.write("This page contains temperature, humidity, and wind speed plots that were made from the Delhi weather database.")

    # Mean temperature plot
    st.subheader("Mean Temperature Over the Years")
    mean_temp_overyears(data)

    # Humidity plot
    st.subheader("Humidity Over the Years")
    humidity_overyears(data)

    # Wind speed plot
    st.subheader("Wind Speed Over the Years")
    wind_speed_overyears(data)

#PAGE 2*************************************************************************

def page2():
    st.header("Historical Observations on AQI of TT Nagar Bhopal-")
    st.write("This Page contains the plots which describe the historical changes in AQI of TT Nagar Bhopal and surrounding areas.")

    #AQI over time
    st.subheader("AQI Overtime")
    aqi_otm(train)
    #AQI over the years
    st.subheader("AQI Over the years")


#PAGE 3*************************************************************************

def page3():
    st.header("Page 3")
    st.write("This is the content of Page 3")

#AQI Data**************************************************************************************************************************************************************
@st.cache_data
def aqi_otm(train):
  figure = px.line(train, x="Date",
                 y="AQI No.",
                 title='AQI over time')
  st.plotly_chart(figure)

@st.cache_data
def aqi_oty(train):
  plt.style.use('fivethirtyeight')
  fig, ax = plt.subplots(figsize=(15, 10))
  plt.title("AQI Change in Bhopal(TT Nagar) Over the Years")
  sns.lineplot(data = train, x='month', y='AQI No.', hue='year')
  st.pyplot(fig)

#Weather Data**********************************************************************************************************************************************************
@st.cache_data
def mean_temp_overyears(data):
    plt.style.use('fivethirtyeight')
    fig, ax = plt.subplots(figsize=(15, 5))
    plt.title("Temperature Change in Delhi Over the Years")
    sns.set(rc={'text.color': 'white'})
    sns.lineplot(data=data, x='month', y='meantemp', hue='year')
    st.pyplot(fig)

@st.cache_data
def wind_speed_overyears(data):
    plt.style.use('fivethirtyeight')
    fig, ax = plt.subplots(figsize=(15, 5))
    plt.title("Humidity Change in Delhi Over the Years")
    sns.set(rc={'text.color': 'white'})
    sns.lineplot(data=data, x='month', y='humidity', hue='year')
    st.pyplot(fig)

@st.cache_data
def humidity_overyears(data):
    plt.style.use('fivethirtyeight')
    fig, ax = plt.subplots(figsize=(15, 5))
    plt.title("Wind speed Change in Delhi Over the Years")
    sns.set(rc={'text.color': 'white'})
    sns.lineplot(data=data, x='month', y='wind_speed', hue='year')
    st.pyplot(fig)

# Load data
data = load_data()
train = load_train()

# Sidebar navigation
page = st.sidebar.selectbox("Select Page:", ["Delhi Weather historical data", "AQI Historical data", "Page 3"])

# Render selected page
if page == "Page 1":
    page1(data)
elif page == "Page 2":
    page2()
elif page == "Page 3":
    page3()
